
## Vector Data Types 

Vector data structures represent specific features on, above, or below the Earth’s surface, and assign attributes to those features. Vectors are composed of discrete geometric locations (x, y values) known as vertices that define the shape of the spatial object. The fundamental primitive is a point.  Once points are entered and geometric lines are created, topology must by “built”. 
#### Topology: 
Involves calculating and encoding the relationships between the points, lines and areas.  There are 4 kinds of realtionships to encode.

##### Adjacency – which polygons are next to each other? Left and right sides of a line or polygon edge.
##### Connectivity – which lines connect to which other lines? Linkages among spatial objects.
##### Containment – which features are located within another feature? (i.e. islands)
##### Coincidence – which features occupy the same locations?

All spatial entities are decomposed and represented in the form of the three basic graphic elements (point, line, polygon).  Very large vector databases have been built for different purposes. Vector tends to dominate in transportation, utility, marketing applications while both raster & vector are both used in resource management applications.

#### Why Topology is important

Topology greatly enhances analysis.
Topological relationships allow these questions to be explored (without it they couldn’t be)
> • Which parcels are adjacent to which? – Adjacency analyses are simply a database table lookup.
> • Which roads are connected to each other for routing?
> • Which fire service areas are adjacent to each other for mutual aid?
> • Which streams are in which watershed?
> • Which polygons of one layer intersect with polygons from another layer (a type of overlay analysis)?– Involves polygon coincidence

The organization of these vertices determines which of the three basic types of vector structures that we are working with: point, line or polygon. 

### Points as XY Coordinates

Vector points are individual X,Y locations. In many cases they are a longitude and latitude but could also be displayed as any number of different coordiante systems (UTM, State Plane, etc.).  As well, they are associated with a spatial reference frame or datum.

Scale Effects: When features are too small to be represented as polygons, points can be used. For example, you may not be able to see the city boundary at small scales. In this case, maps often use points to display locations of cities.

### Lines Connect Vertices

Lines are composed of many (at least 2) points that are connected. Vector lines connect nodes with vertices that generate the path of the line between two nodes. This line is composed of a series of segments, each “bend” in the road or stream represents a vertex that has been defined by a specific x, y location.B asically, by connecting the vertices in a set order, the line gains direction and a left and right hand side, all of which can be leveraged by the data scienctist to explore relationships of topology and association. 

Lines usually represent features that are linear in nature. For example, maps show rivers, roads, powerlines, and pipelines as vector lines. Lines on maps often provide further details about the represented objects through symbolization, such as a busier highway having thicker lines than a private road.

Networks are special line data sets consisting of a system of interconnected linear features through which resources are transported or communication is achieved. The network data model is an abstract representation of the components and characteristics of real-world network systems. These linear networks are topologically connected elements consisting of junctions as well as turns. A network model can be defined as a line graph, which is composed of links representing linear channels of flow and nodes representing their connections (Lupien et al.,1987).

One major application of network analysis is found in transportation planning, where the issue might be to find paths corresponding to certain criteria, (find the shortest or least cost path) or find all locations within a given travel zone (distance or time) from a specified origin. 

### Polygons Connect Vertices and Close Them

A polygon consists of 3 or more vertices that are connected and closed. When you join a set of vertices in a particular order and close it, this is a vector polygon feature. In order to create a polygon, the first and last coordinate pair are the same. There are rules, called planar enforcement, that state that polygons of differing types cannot overlap, and that lines must be split at intersections.

Every piece of a polygon's boundary line is a common boundary between two areas. The stretch of common boundary between two junctions (nodes) has various names.
    Edge: favored by graph theorists, “vertex” for the junctions
    Chain: word officially sanctioned by the US National Standard
    Arc: used by several systems
Arcs have attributes which identify polygons on either side. These are referred to as “left” and “right” by reference to the sequence (or direction)in which the arc is coded.

Polygons are used to show boundaries and have an area and perimeter. For example, a building footprint has a square footage and agricultural fields have acreage.

### Vector Data Advantages
#### Vector data has some important advantages:

The geometry itself contains information about what the dataset creator thought was important.
The geometry structures hold information in and of themselves - why choose point over polygon, for instance?
Each geometry feature can carry multiple attributes instead of just one, e.g. cities can have name, country, population, etc.

#### The downsides of vector data include:

Potential bias in datasets - what didn’t get recorded?
Calculations involving multiple vector layers need to do math on the geometry as well as the attributes, so can be slow.

## GeoSeries
A GeoSeries (or compound object) is essentially a vector where each entry in the vector is a set of shapes corresponding to one observation. An entry may consist of only one shape (like a single polygon) or multiple shapes that are meant to be thought of as one observation (like the many polygons that make up the State of Hawaii or a country like Indonesia). -- From: [Geopanda Documentation](http://geopandas.org/data_structures.html)

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame

First, let's define a simple dictionary of lists.

In [ ]:
data = {'name': ['a', 'b', 'c'],
        'lat': [34, 30, 32],
        'lon': [-118, -120, -123]}

Now create a list of Point shapely objects out of X & Y coordinate lists

In [ ]:
geometry = [Point(xy) for xy in zip(data['lon'], data['lat'])]

Now create a GeoSeries where we explicitly define the index values.

In [ ]:
series = GeoSeries(geometry, index=data['name'])
series

Additionally, a GeoSeries (and a GeoDataFrame) can store the CRS implicitly associated with the geometry column. This is useful and an essential piece of spatial metadata that can be used for transformation (reprojection) to another CRS. 
Let’s assign the CRS to Web Mercator

In [ ]:
# series.crs = {'init': 'epsg:3857'} this operation syntax is depreciated and is in the process of being removed.
series.crs = ('epsg:3857')

Now let's plot the points

In [ ]:
series.plot(marker='+', color='blue', markersize=10, figsize=(4, 4))
plt.ylim([25, 40])
plt.xlim([-125, -115]);

##  GeoDataFrames

A GeoDataFrame is a tabular data structure that contains a GeoSeries.

The most important property of a GeoDataFrame is that it always has one GeoSeries column that holds a special status. This GeoSeries is referred to as the GeoDataFrame’s “geometry”. When a spatial method is applied to a GeoDataFrame (or a spatial attribute like area is called), this command will always act on the “geometry” column.

The “geometry” column – no matter its name – can be accessed through the geometry attribute (gdf.geometry), and the name of the geometry column can be found by typing gdf.geometry.name.

A GeoDataFrame may also contain other columns with geometrical (shapely) objects, but only one column can be the active geometry at a time. To change which column is the active geometry column, use the set_geometry method. -- From [GeoPanda Documentation](http://geopandas.org/data_structures.html)

Let's look at creating a GeoDataFrame instead of a simple GeoSeries

We will start with reusing our data directory but this time creating a dataframe with it.

In [ ]:
df = pd.DataFrame(data)
df

Now we use the DataFrame and the “list-of-shapely-Point-objects” approach to create a GeoDataFrame. Note the use of two DataFrame attribute columns, which are effectively just two simple Pandas Series.

In [ ]:
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
gdf = GeoDataFrame(df, geometry=geometry)

There’s nothing new to visualize, but this time we’re using the plot method from a GeoDataFrame, not from a GeoSeries. They’re not exactly the same thing under the hood.

In [ ]:
gdf.plot(marker='*', color='purple', markersize=10, figsize=(3, 3));

# Save Your Notebook
## Then Notebook Menu:  File > Close and Halt